# Publications markdown generator for academicpages

Takes a TSV of publications with metadata and converts them for use with [academicpages.github.io](academicpages.github.io). This is an interactive Jupyter notebook ([see more info here](http://jupyter-notebook-beginner-guide.readthedocs.io/en/latest/what_is_jupyter.html)). The core python code is also in `publications.py`. Run either from the `markdown_generator` folder after replacing `publications.tsv` with one containing your data.

TODO: Make this work with BibTex and other databases of citations, rather than Stuart's non-standard TSV format and citation style.


## Data format

The TSV needs to have the following columns: pub_date, title, venue, excerpt, citation, site_url, and paper_url, with a header at the top. 

- `excerpt` and `paper_url` can be blank, but the others must have values. 
- `pub_date` must be formatted as YYYY-MM-DD.
- `url_slug` will be the descriptive part of the .md file and the permalink URL for the page about the paper. The .md file will be `YYYY-MM-DD-[url_slug].md` and the permalink will be `https://[yourdomain]/publications/YYYY-MM-DD-[url_slug]`

This is how the raw file looks (it doesn't look pretty, use a spreadsheet or other program to edit and create).

In [1]:
!cat publications.csv

pub_date,title,venue,vol,pages,year,author1,author2,mypos,citation,url_slug,paper_url,excerpt
2015-02-01,Quantum interference in time-delayed nonsequential double ionization,Phys. Rev. A,92,023421,2015,0,C. Figueira de Morisson Faria,1,"A. S. Maxwell and C. Figueira de Morisson Faria. Phys. Rev. A, 92, 23421 (2015).",Quantum interference in time-delayed nonsequential double ionization,https://journals.aps.org/pra/abstract/10.1103/PhysRevA.92.023421,
2015-08-01,A Systematic Study of Interference Effects in Non-Sequential Double Ionisation,J. Phys. Conf. Ser.,635,92136,2015,0,C. Figueira de Morisson Faria,1,"A. S. Maxwell and C. Figueira de Morisson Faria. J. Phys.: Conf. Ser., 635, 092136 (2015).",A Systematic Study of Interference Effects in Non-Sequential Double Ionisation,https://iopscience.iop.org/article/10.1088/1742-6596/635/9/092136/meta,
2015-08-01,On properties of Karlsson Hadamards and sets of mutually unbiased bases in dimension six,"Linear Algebra and Its Applications,",466,

## Import pandas

We are using the very handy pandas library for dataframes.

In [2]:
import pandas as pd

## Import TSV

Pandas makes this easy with the read_csv function. We are using a TSV, so we specify the separator as a tab, or `\t`.

I found it important to put this data in a tab-separated values format, because there are a lot of commas in this kind of data and comma-separated values can get messed up. However, you can modify the import statement, as pandas also has read_excel(), read_json(), and others.

In [3]:
publications = pd.read_csv("publications.csv", sep="\t", header=0)
publications


,"pub_date,title,venue,vol,pages,year,author1,author2,mypos,citation,url_slug,paper_url,excerpt"
0,"2015-02-01,Quantum interference in time-delaye..."
1,"2015-08-01,A Systematic Study of Interference ..."
2,"2015-08-01,On properties of Karlsson Hadamards..."
3,"2016-08-24,Controlling Below-Threshold Nonsequ..."
4,"2017-08-22,Coulomb-corrected quantum interfere..."
5,"2018-01-26,Analytic quantum-interference condi..."
6,"2018-05-23,Coulomb-free and Coulomb-distorted ..."
7,"2018-12-20,Treating Branch Cuts in Quantum Tra..."
8,"2019-10-14,Symphony on Strong Field Approximat..."
9,"2020-01-01,Polarisation Effects in Above-Thres..."


## Escape special characters

YAML is very picky about how it takes a valid string, so we are replacing single and double quotes (and ampersands) with their HTML encoded equivilents. This makes them look not so readable in raw format, but they are parsed and rendered nicely.

In [4]:
html_escape_table = {
    "&": "&amp;"
    ,'"': "&quot;"
    ,"'": "&apos;"
    #,":": "&colon;"
}

def html_escape(text):
    """Produce entities within text."""
    return "".join(html_escape_table.get(c,c) for c in text)

In [5]:
html_escape(":")

':'

publications## Creating the markdown files

This is where the heavy lifting is done. This loops through all the rows in the TSV dataframe, then starts to concatentate a big string (```md```) that contains the markdown for each type. It does the YAML metadata first, then does the description for the individual page.

In [6]:
import os
#Add option to perge folder here, to prevent duplicates
#solution is only valid for unix
os.system("rm ../_publications/*")

for row, item in publications.iterrows():
    
    md_filename = str(item.pub_date) + "-" + item.url_slug + ".md"
    html_filename = str(item.pub_date) + "-" + item.url_slug
    year = item.pub_date[:4]
    
    ## YAML variables
    
    md = "---\ntitle: \""   + item.title + '"\n'
    
    md += """collection: publications"""
    
    md += """\npermalink: \" /publication/""" + html_filename + "\""
    
    if len(str(item.excerpt)) > 5:
        md += "\nexcerpt: '" + html_escape(item.excerpt) + "'"
    
    md += "\ndate: " + str(item.pub_date) 
    
    md += "\nvenue: '" + html_escape(item.venue) + "'"
    
    if len(str(item.paper_url)) > 5:
        md += "\npaperurl: '" + item.paper_url + "'"
    
    md += "\ncitation: '" + html_escape(item.citation) + "'"
    
    md += "\n---"
    
    ## Markdown description for individual page
        
    if len(str(item.excerpt)) > 5:
        md += "\n" + html_escape(item.excerpt) + "\n"
    
    #if len(str(item.paper_url)) > 5:
    #    md += "\n[Download paper here](" + item.paper_url + ")\n" 
        
    #md += "\n" + item.citation
    
    md_filename = os.path.basename(md_filename)
       
    with open("../_publications/" + md_filename, 'w') as f:
        f.write(md)

These files are in the publications directory, one directory below where we're working from.

In [7]:
!ls ../_publications/

'2015-02-01-Quantum interference in time-delayed nonsequential double ionization.md'
'2015-08-01-A Systematic Study of Interference Effects in Non-Sequential Double Ionisation.md'
'2015-08-01-On properties of Karlsson Hadamards and sets of mutually unbiased bases in dimension six.md'
'2016-08-24-Controlling Below-Threshold Nonsequential Double Ionization via Quantum Interference.md'
'2017-08-22-Coulomb-corrected quantum interference in above-threshold ionization: Working towards multi-trajectory electron holography.md'
'2018-01-26-Analytic quantum-interference conditions in Coulomb corrected photoelectron holography.md'
'2018-05-23-Coulomb-free and Coulomb-distorted recolliding quantum orbits in photoelectron holography.md'
'2018-12-20-Treating Branch Cuts in Quantum Trajectory Models for Photoelectron Holography.md'
'2019-10-14-Symphony on Strong Field Approximation.md'
'2020-01-01-H. P. Kang, A. S. Maxwell et al. Holographic detection of parity in atomic and molecular orbitals Phys. 

## Create tex list of publications for latex CVpublications.iterrows()

In [8]:
tex_filename = "PublicationListGen.tex"

me= r'\underline{A. S. Maxwell}'


with open("../../../CV/gitCV/" + tex_filename, 'w') as f:
    tex = r'\begin{etaremune}%'
    tex+='\n'
    tex+= r'\itemsep0em'
    tex+='\n\n'
    f.write(tex)
    
    for row, item in (publications[::-1]).iterrows():
        tex = r'\item '
        #Put first author special command for if it is me
        if(item.mypos==1):
            if(item.author1=='0'):
                tex+=me + ', '
            else:
                tex+=item.author1 + ', '+ me+ ' [joint first author] et al.'
        else:
            tex+=item.author1 + ', '
        #Put second author special command for if it is me
        if(item.mypos==2):
            tex+=me + ' et al.'
        else:
            if(item.author2!='0'):
                tex+=item.author2
                if(item.mypos==1):
                    tex+=' et al.'
        #if I am middle author
        if(item.mypos==0):
             tex+=r' [et al.\ including '+me+' ]'
        #last author case
        if(item.mypos==-1):
             tex+=' et al. and '+me+' '
        
        tex+='\n'            
        #Put the title in italics
        tex+= r' \emph{' + item.title + '}'
        tex+='\n'
        #put journal head
        tex+= item.venue + ' '
        tex+= r'\href{' + item.paper_url+'}'
        tex+='\n'
        tex+= r'{\textbf{'
        tex+= str(item.vol) + r'}'
        if(item.year!=0):
            tex+=', '
        if(item.pages!=0):
            tex+= str(item.pages)
        tex+= r'} '
        if(item.year!=0):
            tex+='(' + str(int(item.year)) + ')'
        tex+= '\n\n'
        f.write(tex)
    
    tex='\end{etaremune}'
    f.write(tex)